In [1]:
# Prevent pycache from being generated while developing
import sys
sys.dont_write_bytecode=True

In [2]:
# Import required packages
import ee
from pprint import pprint

In [3]:
# Import local files
import config
import geelayers
import cities
import helpers
import super_rasters

GHSL PROJ: EPSG:3857 [100, 0, -18041000, 0, -100, 9000000]


In [17]:
# Reload local files while developing
import importlib
# importlib.reload(cities)
importlib.reload(geelayers)
# importlib.reload(config)
# importlib.reload(helpers)
# importlib.reload(super_rasters)

GHSL PROJ: EPSG:3857 [100, 0, -18041000, 0, -100, 9000000]


<module 'geelayers' from '/home/weiqi_tori/GitHub/wri/urban_extent/gee/geelayers.py'>

In [4]:
# Get list of image collections in a folder
def get_collections(folder):
    collections = ee.data.getList({'id': folder})
    return collections


collections = get_collections(config.ROOT)


## Select Image collection
### Count the number of images in the collection
size = len(collections)
print(size)

### Extract image names as a list
collection_ids = [collection['id'] for collection in collections]
pprint(collection_ids)

13
['projects/earthengine-legacy/assets/projects/wri-datalab/cities/urban_land_use/data/test_tori_Apr2024/GHSL_BUthresh10pct_JRCs_50compare_L2_20_wP_2020',
 'projects/earthengine-legacy/assets/projects/wri-datalab/cities/urban_land_use/data/test_tori_Apr2024/GHSL_BUthresh10pct_JRCs_50compare_L2_20_wP_2020_circle',
 'projects/earthengine-legacy/assets/projects/wri-datalab/cities/urban_land_use/data/test_tori_Apr2024/builtup_density_JRCs_1000',
 'projects/earthengine-legacy/assets/projects/wri-datalab/cities/urban_land_use/data/test_tori_Apr2024/builtup_density_JRCs_50compare_L1',
 'projects/earthengine-legacy/assets/projects/wri-datalab/cities/urban_land_use/data/test_tori_Apr2024/builtup_density_JRCs_50compare_L2',
 'projects/earthengine-legacy/assets/projects/wri-datalab/cities/urban_land_use/data/test_tori_Apr2024/builtup_density_JRCs_50compare_L2_100',
 'projects/earthengine-legacy/assets/projects/wri-datalab/cities/urban_land_use/data/test_tori_Apr2024/builtup_density_JRCs_50compar

In [ ]:
# Create map
collection = 'projects/earthengine-legacy/assets/projects/wri-datalab/cities/urban_land_use/data/test_tori_Apr2024/builtup_density_JRCs_1000'

#importlib.reload(geelayers)
geelayers.interactive_map(collection)

In [8]:
size = geelayers.GHSL_2020.size().getInfo()
print(f"Image Collection Size: {size}")
print(f'DEST: {config.IC_ID}')
#pprint(geelayers.CITY_DATA.sort('UC_NM_MN').aggregate_array('UC_NM_MN').getInfo())

Image Collection Size: 4241
DEST: projects/wri-datalab/cities/urban_land_use/data/test_tori_Apr2024/builtup_density_JRCs_50compare_L2_20_wP


In [9]:
geelayers.CITY_DATA = geelayers.CITY_DATA.sort('P15', False)
OFFSET = config.OFFSET
LIMIT = config.LIMIT

if OFFSET and LIMIT:
    LIMIT = LIMIT+OFFSET
IDS = geelayers.CITY_DATA.aggregate_array('ID_HDC_G0').getInfo()[OFFSET:LIMIT]

pprint(len(IDS))

12844


In [ ]:
# Map the function over the feature collection
geelayers.CITY_DATA = geelayers.CITY_DATA.map(helpers.replace_geometry)

In [ ]:
# Get list of cities
CITIES_LIST = cities.id_hdc_g0_250[1:2]
CITIES_LIST

In [ ]:
# Calculate Urban Extent for a list of cities
super_rasters.get_urban_extents(IDS, CITIES_LIST)

In [ ]:
# Filter to limit cities to be run to those not already in the output ImageCollection.
COMPLETED_IDS = ee.ImageCollection(config.IC_ID).aggregate_array('ID_HDC_G0')
COMPLETED_FILTER = ee.Filter.And(ee.Filter.inList('ID_HDC_G0', COMPLETED_IDS), ee.Filter.equals('builtup_year', config.mapYear))
COMPLETED_CITIES_LIST = ee.ImageCollection(config.IC_ID).filter(COMPLETED_FILTER).aggregate_array('ID_HDC_G0')

if config.USE_COMPLETED_FILTER:
    geelayers.CITY_DATA = geelayers.CITY_DATA.filter(ee.Filter.inList('ID_HDC_G0', COMPLETED_CITIES_LIST).Not())
else:
    geelayers.CITY_DATA = geelayers.CITY_DATA

pprint(geelayers.CITY_DATA.aggregate_array('UC_NM_MN').getInfo())
pprint(geelayers.CITY_DATA.aggregate_array('GRGN_L2').getInfo())
pprint(geelayers.CITY_DATA.aggregate_array('ID_HDC_G0').getInfo())